In [113]:
from __future__ import print_function

import pandas as pd
import os 
import boto3
import json
from datetime import datetime
import boto3 

ts = datetime.now().strftime("%Y-%m-%d")
    
root_folder = os.getcwd()
root_folder = "/home/paperspace/data"
print(root_folder)
image_root = '/home/paperspace/data/images/niihau'
supervisely_folder_root = '/home/paperspace/.supervisely-agent/rQnKzzBnUneHRgIp0NOXDRU2URPEGPC5/import/marine_debris_ann_review'

boto3.setup_default_session(profile_name='hawaii')
s3_client = boto3.client('s3')
s3_resource = boto3.resource('s3')
bucket_name = 'hawaii-marine-debris'


/home/paperspace/data


In [114]:
s3_resource.Bucket(bucket_name).download_file("ml/result/annotations_to_be_reviewed.csv","{}/annotations_to_be_reviewed.csv".format(root_folder))

In [115]:
ann = pd.read_csv("{}/annotations_to_be_reviewed.csv".format(root_folder))
ann = ann.drop('Unnamed: 0',axis=1)
ann[:5]

,s3_key,x1,y1,x2,y2,label,priority_tag,tag,confidence_label
0,niihau/jpg/1_1130_16_27.jpg,774.0,288.0,814.0,329.0,Plastic,high,false_positive,medium
1,niihau/jpg/1_1130_16_27.jpg,775.0,289.0,814.0,328.0,Processedwood,high,false_positive,low
2,niihau/jpg/1_1130_16_27.jpg,784.0,293.0,827.0,334.0,Plastic,high,false_positive,low
3,niihau/jpg/1_1130_16_27.jpg,774.0,289.0,814.0,329.0,Foam,high,false_positive,low
4,niihau/jpg/1_1378_24_28.jpg,741.0,660.0,781.0,700.0,Buoys,high,false_positive,medium


In [116]:

s3_resource.Bucket(bucket_name).download_file("niihau/final_marine_debris_database_NI_UTM4N_boxes.geojson", "{}/final_marine_debris_database_NI_UTM4N_boxes.geojson".format(root_folder))
s3_resource.Bucket(bucket_name).download_file("niihau/niihau_tileindex.geojson", "{}/niihau_tileindex.geojson".format(root_folder))

s3_resource.Bucket(bucket_name).download_file("niihau/new_annotations.csv", "{}/new_annotations.csv".format(root_folder))

# s3_resource.Bucket(bucket_name).download_file("niihau/annotations.csv", "{}/niihau/annotations.csv".format(root_folder))


In [117]:
s3_resource.Bucket(bucket_name).download_file("ml/v2/data/labels.csv", "{}/niihau/labels.csv".format(root_folder))
# s3://hawaii-marine-debris/

In [118]:
s3_resource.Bucket(bucket_name).download_file("niihau/deduped_annotations.csv", "{}/niihau/annotations.csv".format(root_folder))
labels_df = pd.read_csv("{}/niihau/labels.csv".format(root_folder), encoding='utf-8',names=['label','id'])
labels_df

,label,id
0,Bicycle,0
1,Boat,1
2,Surfboard,2
3,Bottle,3
4,Plastic bag,4
5,Wheel,5
6,Tire,6
7,Buoys,7
8,Cloth,8
9,Foam,9


In [119]:

label_lookup = labels_df.set_index('id').T.to_dict('records')[0]
label_lookup 

{0: 'Bicycle',
 1: 'Boat',
 2: 'Surfboard',
 3: 'Bottle',
 4: 'Plastic bag',
 5: 'Wheel',
 6: 'Tire',
 7: 'Buoys',
 8: 'Cloth',
 9: 'Foam',
 10: 'Line',
 11: 'Metal',
 12: 'Net',
 13: 'Plastic',
 14: 'Processedwood',
 15: 'Other'}

In [120]:
# if only_new:
#     annotations_df = pd.read_csv("{}/new_annotations.csv".format(root_folder), encoding='utf-8')
#     annotations_df = annotations_df.rename(columns={'image_s3_key':'s3_key'})

# else:
#     annotations_df = pd.read_csv("{}/niihau/annotations.csv".format(root_folder), encoding='utf-8')

#     annotations_df['label'] = annotations_df['label'].replace({'B':'Buoys','C': 'Cloth','F':'Foam','L':'Line','M':'Metal','N':'Net','P':'Plastic','T':'Tire','W':'Processedwood','V':'Boat'})


In [121]:
import random
def random_color():
    r = lambda: random.randint(0,255)
    return '#%02X%02X%02X' % (r(),r(),r())
random_color()

'#3259FE'

In [122]:
for i, v in labels_df.iterrows():
    print(v.label)
    break

Bicycle


In [139]:

if not os.path.isdir("{}/ann".format(image_root)):
    os.mkdir("{}/ann".format(image_root))
niihau_sup_root = supervisely_folder_root+"/niihau"
niihau_img_root = "{}/img".format(niihau_sup_root)


# os.mkdir(niihau_sup_root)
os.makedirs(niihau_img_root)
os.makedirs("{}/ann".format(niihau_sup_root))

In [140]:
label_list = labels_df['label'].unique().tolist()
label_list

['Bicycle',
 'Boat',
 'Surfboard',
 'Bottle',
 'Plastic bag',
 'Wheel',
 'Tire',
 'Buoys',
 'Cloth',
 'Foam',
 'Line',
 'Metal',
 'Net',
 'Plastic',
 'Processedwood',
 'Other']

In [141]:
def get_image(s3_key):
    local_image_path = "{}/{}".format(image_root, os.path.basename(s3_key))
    if not os.path.isfile(local_image_path):
        s3_resource.Bucket(bucket_name).download_file(s3_key, local_image_path)
    return local_image_path

from PIL import Image

def get_width_height(s3_key):
    img_path = get_image(s3_key)
    im = Image.open(img_path)
    width, height = im.size
    return width, height
width, height = get_width_height("niihau/jpg/1_1920_04_07.jpg")
print(width, height)

1000 1000


In [142]:
ann['label'] = ann['label'].replace({'wood':'Processedwood','O':'Other'})
ann['tag'].unique()

array(['false_positive', 'false_negative'], dtype=object)

In [143]:
for label in ann['label'].unique().tolist():
    if label not in label_list:
        print("{} not exists".format(label))

In [144]:
print(len(ann[ann['confidence_label'].isin(['medium','high','manual'])]))

ann = ann[ann['confidence_label'].isin(['medium','high','manual'])]

342


In [145]:
from shutil import copyfile

object_tags = set()
all_image_tags = set()
def generate_image_json(image_name, island, list_of_debris):
    width, height = get_width_height(image_name)
    random_reviewer ="reviewer_{}".format(random.randint(1, 5))
    image_tags = set(["needs_review",random_reviewer])
    
    for d in list_of_debris:
        for t in d["tags"]:
            image_tags.add(t)
            all_image_tags.add(t)
        
    return {
        "description": "{}_{}".format(island,image_name),
        "name": "{}_{}".format(island,image_name),
        "size": {
            "width": 1024,
            "height": 1024
        },
        "tags": list(image_tags),
        "objects": list_of_debris
        }
def generate_obj_json(debris_row):
    
    
    this_object_tags = [debris_row.tag, "priority_{}".format(debris_row.priority_tag)]
    if debris_row.confidence_label == 'confidence_manual':
        this_object_tags.add("confidence_{}".format(debris_row.confidence_label))
    for t in this_object_tags:
        object_tags.add(t)
    return {
      "description": debris_row.label,
      "tags": this_object_tags,
      "bitmap": None,
      "classTitle": debris_row.label,
      "points": {
        "exterior": [
          [
            debris_row.x1,
            debris_row.y1
          ],
          [
            debris_row.x2,
            debris_row.y2
          ]
        ],
        "interior": []
      }
    }

for key, grouped in ann.groupby('s3_key'):
    list_of_objs = []
    for i, val in grouped.iterrows():
        list_of_objs.append(generate_obj_json(val))
    json_per_image = generate_image_json(key, 'niihau', list_of_objs)
#     print(json.dumps(json_per_image, indent=4))
    src_img_path = "{}/{}".format(image_root, os.path.basename(key))
    dst_img_path = "{}/{}".format(niihau_img_root, os.path.basename(key))
    copyfile(src_img_path, dst_img_path)
    json_file_name = "{}/ann/{}".format(niihau_sup_root, os.path.basename(key).replace(".jpg",".json"))
    with open(json_file_name, 'w') as outfile:
        json.dump(json_per_image, outfile)


In [146]:
o_tags = list(object_tags)+["urgent","risk","low_risk"]
print(o_tags)
i_tags = set(["inconclusive","debris_deleted","found_new_debris","needs_review","fixed",'review_done','debris','reviewer_1','reviewer_2','reviewer_3','reviewer_4', 'reviewer_5'])
for tag in all_image_tags:
    i_tags.add(tag)
list(i_tags)

['false_positive', 'false_negative', 'priority_high', 'urgent', 'risk', 'low_risk']


['inconclusive',
 'review_done',
 'reviewer_3',
 'reviewer_4',
 'needs_review',
 'false_negative',
 'debris_deleted',
 'priority_high',
 'fixed',
 'reviewer_5',
 'found_new_debris',
 'reviewer_2',
 'debris',
 'false_positive',
 'reviewer_1']

In [147]:
i_tags

{'debris',
 'debris_deleted',
 'false_negative',
 'false_positive',
 'fixed',
 'found_new_debris',
 'inconclusive',
 'needs_review',
 'priority_high',
 'review_done',
 'reviewer_1',
 'reviewer_2',
 'reviewer_3',
 'reviewer_4',
 'reviewer_5'}

In [148]:
d = {
    "tags_objects": o_tags,
    "tags_images": list(i_tags),
    "classes":[]
}

for i, v in labels_df.iterrows():
    color_dict = {"title":v.label,
                  'color':random_color(),
                  'shape':'rectangle'
                  }
    d['classes'].append(color_dict)
    
meta_json_path = "{}/meta.json".format(supervisely_folder_root)
with open(meta_json_path, 'w') as outfile:
    json.dump(d, outfile)


In [112]:
meta_json_path

'/home/paperspace/.supervisely-agent/rQnKzzBnUneHRgIp0NOXDRU2URPEGPC5/import/marine_debris_ann_review/meta.json'

In [78]:
niihau_sup_root

'/home/paperspace/.supervisely-agent/rQnKzzBnUneHRgIp0NOXDRU2URPEGPC5/import/marine_debris_ann_review/niihau'